In [9]:
!pip install langchain_chroma langchain_huggingface langchain python-dotenv chromadb openai tiktoken pypdf langchain_openai langchain-community

In [11]:
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

load_dotenv()

embedding  = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


doc1 = Document(
  page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
  metadata={"team": "Royal Challengers Bangalore"}
)
doc2 = Document(
  page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  metadata={"team": "Mumbai Indians"}
)
doc3 = Document(
  page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
  metadata={"team": "Chennai Super Kings"}
)
doc4 = Document(
  page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
  metadata={"team": "Mumbai Indians"}
)
doc5 = Document(
  page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
  metadata={"team": "Chennai Super Kings"}
)

docs = [doc1, doc2, doc3, doc4, doc5]



In [14]:
vector_store = Chroma(embedding_function=embedding,
                      persist_directory='/content/drive/MyDrive/ML-AI/GenAI/LangChain/code-base/vector-store/my_chroma_db',
                      collection_name='sample')

In [15]:
vector_store.add_documents(docs)

['cbcc67d0-2b93-453d-a322-288e8f96b791',
 '1605c44f-a198-4acd-b47c-665d90a31882',
 '473f582b-f6d4-47b7-a973-317cce8bffa7',
 '88f49fa4-2a19-4f34-838d-efa4342e831e',
 '0ec32d9f-4755-461b-992d-32b73749b658']

In [17]:
vector_store.get(include = ['embeddings', 'metadatas', 'documents'])

{'ids': ['cbcc67d0-2b93-453d-a322-288e8f96b791',
  '1605c44f-a198-4acd-b47c-665d90a31882',
  '473f582b-f6d4-47b7-a973-317cce8bffa7',
  '88f49fa4-2a19-4f34-838d-efa4342e831e',
  '0ec32d9f-4755-461b-992d-32b73749b658'],
 'embeddings': array([[ 0.00994728,  0.06914341, -0.05147117, ..., -0.03543337,
          0.0128481 ,  0.01248289],
        [ 0.00127744,  0.03129853, -0.02375378, ..., -0.00518361,
         -0.03280616,  0.02737717],
        [-0.10265916,  0.02650811,  0.02271503, ..., -0.03359746,
         -0.07984947, -0.01507707],
        [ 0.02123393, -0.02468549, -0.04494375, ..., -0.10995814,
          0.00572561,  0.09915381],
        [ 0.01873977,  0.04382848, -0.04304255, ..., -0.0780162 ,
         -0.07840683, -0.0030419 ]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [21]:
# Similarity search
vector_store.similarity_search(
    query = 'Who among these are a bowler?',
    k = 1 # how many similar data to show
)

[Document(id='88f49fa4-2a19-4f34-838d-efa4342e831e', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.')]

In [25]:
# Similarity search with score
vector_store.similarity_search_with_score(
    query = 'Who among these are a bowler?',
    k = 1 # how many similar data to show
)

[(Document(id='88f49fa4-2a19-4f34-838d-efa4342e831e', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.9693600535392761)]

In [26]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(id='473f582b-f6d4-47b7-a973-317cce8bffa7', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.8436007499694824),
 (Document(id='0ec32d9f-4755-461b-992d-32b73749b658', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.890937328338623)]

In [27]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='cbcc67d0-2b93-453d-a322-288e8f96b791', document=updated_doc1)


In [28]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['cbcc67d0-2b93-453d-a322-288e8f96b791',
  '1605c44f-a198-4acd-b47c-665d90a31882',
  '473f582b-f6d4-47b7-a973-317cce8bffa7',
  '88f49fa4-2a19-4f34-838d-efa4342e831e',
  '0ec32d9f-4755-461b-992d-32b73749b658'],
 'embeddings': array([[-0.00233746,  0.05902082, -0.0477405 , ..., -0.07264049,
          0.00276782, -0.00344082],
        [ 0.00127744,  0.03129853, -0.02375378, ..., -0.00518361,
         -0.03280616,  0.02737717],
        [-0.10265916,  0.02650811,  0.02271503, ..., -0.03359746,
         -0.07984947, -0.01507707],
        [ 0.02123393, -0.02468549, -0.04494375, ..., -0.10995814,
          0.00572561,  0.09915381],
        [ 0.01873977,  0.04382848, -0.04304255, ..., -0.0780162 ,
         -0.07840683, -0.0030419 ]]),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a sin

In [29]:
# delete document
vector_store.delete(ids=['0ec32d9f-4755-461b-992d-32b73749b658'])

In [30]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['cbcc67d0-2b93-453d-a322-288e8f96b791',
  '1605c44f-a198-4acd-b47c-665d90a31882',
  '473f582b-f6d4-47b7-a973-317cce8bffa7',
  '88f49fa4-2a19-4f34-838d-efa4342e831e'],
 'embeddings': array([[-0.00233746,  0.05902082, -0.0477405 , ..., -0.07264049,
          0.00276782, -0.00344082],
        [ 0.00127744,  0.03129853, -0.02375378, ..., -0.00518361,
         -0.03280616,  0.02737717],
        [-0.10265916,  0.02650811,  0.02271503, ..., -0.03359746,
         -0.07984947, -0.01507707],
        [ 0.02123393, -0.02468549, -0.04494375, ..., -0.10995814,
          0.00572561,  0.09915381]]),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to